In [71]:
import pandas as pd

df = pd.read_excel('../数据/MCMB数据.xlsx')
df.query('年 == 2000')['人口密度（人/平方千米）']
# print(test[test.index[0]])

{'年': 2000.0,
 '人口密度（人/平方千米）': 65.02,
 '耕地面积': 112797.0,
 '森林面积': 26981.0,
 '大象致人伤亡数量': 4.0,
 '食草动物致人伤亡数量': 1.0,
 '食肉动物致人伤亡数量': 8.0,
 '营地累计数量和住宿（MMNR内部以及周边地区）': 23.0,
 '犀牛栖息地百分比': 18.0,
 '牛放牧扩大占比': 17.0}

In [115]:
from scipy.optimize import minimize
import numpy as np
import math


def getFun(args):
    k1, k2, k3 = args
    fun1 = lambda x: k3 * x[2]
    fun2 = lambda x: (k1 * x[0] + k2 * x[1]) ** 2 - (k3 * x[2]) ** 2

    f = lambda x: -(fun1(x) / (fun1(x) + fun2(x)) - fun2(x) / (fun1(x) + fun2(x)))
    return f


def getCon(args):
    argsKey = list(args.keys())
    x1min = 0
    x1max = args[argsKey[0]]

    x2min = 0
    x2max = args[argsKey[1]]

    x3min = 0
    x3max = args[argsKey[2]]
    cons = (
        {'type': 'ineq', 'fun': lambda x: x[0] - x1min},
        {'type': 'ineq', 'fun': lambda x: -x[0] + x1max},

        {'type': 'ineq', 'fun': lambda x: x[1] - x2min},
        {'type': 'ineq', 'fun': lambda x: -x[1] + x2max},

        {'type': 'ineq', 'fun': lambda x: x[2] - x3min},
        {'type': 'ineq', 'fun': lambda x: -x[2] + x3max},

        {'type': 'ineq', 'fun': lambda x: (args[argsKey[1]] - x[1]) / (args[argsKey[0]] - x[0]) -
                                          args[argsKey[3]] / (2000 - 1948 + 1)},
    )
    return cons


# k1, k2, k3
argsK = (1, 1, 1)

search2000 = df.query('年 == 2000')
year2000 = search2000.loc[search2000.index.values[0]].to_dict()
sum19482000biai = 0
for i, row in df.iterrows():
    sum19482000biai += row['耕地面积'] / row['人口密度（人/平方千米）']

argsCon = {
    '第2000年人口': year2000['人口密度（人/平方千米）'],
    '第2000年耕地': year2000['耕地面积'],
    '第2000年森林': year2000['森林面积'],
    'sum1948-2000(bi/ai)': sum19482000biai
}
print(argsCon)

# 变量初始值
x0 = np.asarray((1, 1, 1))
res = minimize(getFun(argsK), x0, constraints=getCon(argsCon))

print(res)

{'第2000年人口': 65.02, '第2000年耕地': 112797.0, '第2000年森林': 26981.0, 'sum1948-2000(bi/ai)': 94926.16360342082}


D:/Users/th/AppData/Local/Temp/xpython_73860/3469422590.py:35: RuntimeWarning: divide by zero encountered in double_scalars
  {'type': 'ineq', 'fun': lambda x: (args[argsKey[1]] - x[1]) / (args[argsKey[0]] - x[0]) -
D:\Users\th\.conda\envs\py310\lib\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


     fun: 1.0689859184561579
     jac: array([ 0.00426739,  0.00426739, -0.0042928 ])
 message: 'Singular matrix E in LSQ subproblem'
    nfev: 98
     nit: 20
    njev: 20
  status: 5
 success: False
       x: array([ 65.02      ,  72.2280365 , 153.06053328])
